 # Loan Application Chatbot
 This script sets up a chatbot to handle loan application queries using OpenAI's GPT-3.5 and LangChain.

 ## Table of Contents
 1. [Import Libraries](#import-libraries)
 2. [Set Up API Keys](#set-up-api-keys)
 3. [Configure LLM Model](#configure-llm-model)
 4. [Load Document](#load-document)
     1. [Load Document](#load-document)
     2. [Create Searchable Vector Index](#create-searchable-vector-index)
 5. [Initialize LLM](#initialize-llm)
     1. [Customer Service Chain](#customer-service-chain)
     2. [Q & A Chain](#q-a-chain)
     3. [Handle Customer Query](#handle-customer-query)
 6. [Testing the Chatbot](#testing-the-chatbot)

 ## 1. Import Libraries

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

 ## 2. Set Up API Keys

In [ ]:
# OpenAI API key
openai_key  = your_openai_key

 ## 3. Configure LLM Model
 Specify the LLM model to use.

In [ ]:
# LLM model to use
llm_model = "gpt-3.5-turbo-0125"

 ## 4.1 Load Document

 Load the loan applications data. This data will be used to create a searchable vector index.

 For experimentation purposes, we will use a sample dataset with only 3 application records.

In [ ]:
import os
from langchain.document_loaders import CSVLoader

# Get the directory of the current script
script_dir = os.path.dirname(os.path.abspath(__file__))  

# Define the CSV file path
file_path = os.path.join(script_dir, "loan_applications.csv")  

# each row in the CSV is treated as a separate document
loader = CSVLoader(file_path=file_path)
data = loader.load()

 ## 4.2 Create Searchable Vector Index
 Convert text into vector embeddings and create a searchable vector index.

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(openai_api_key=openai_key)

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embedding_model
).from_loaders([loader])

 ## 5. Initialize LLM
 Initialize the LLM with the specified model and API key.

In [ ]:
llm = ChatOpenAI(temperature=0.0, 
                 model=llm_model,
                 openai_api_key=openai_key)

 ## 5.1 Customer Service Chain

In [ ]:
# Prompt template for polite and professional responses
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt_template = """
You are a professional customer service representative for a financial institution. Your tone should always be polite, clear and precise. 

Please only if necessary, retrieve application-related data (such as application status, the reason for decline) to give a complete response.
To do that, you need to ask for application_id first so that you can locate the correct info.

If you don't know the answer, admit it and refer the customer for human support.

Customer Query: "{question}"
"""

custom_prompt = PromptTemplate(input_variables=["question"], 
                               template=prompt_template)

In [ ]:
# Let LLM have memory
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(llm=llm,
                                          max_token_limit=100)

# Customer service chain
customer_service_chain = LLMChain(prompt=custom_prompt, 
                                  llm=llm,
                                  memory=memory)

 ## 5.2 Q & A Chain

In [ ]:
# Combine the LLM and retriever into a question-answering pipeline
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",  # All documents are concatenated into a single string 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs={"document_separator": "<<<<>>>>>"}  # Custom separator
)

In [ ]:
# helper function to extract application id from the query so that Q&A chain can retrieve the relevant information
def extract_application_id(query):
    """
    Function to extract application_id from the query.
    """
    prompt_template = PromptTemplate(
        input_variables=["query"],
        template="Please extract and only return the application ID from the following query: '{query}'. If there is no application id, return 'Missing'."
    )

    application_id_chain = LLMChain(prompt=prompt_template, llm=llm)
    response = application_id_chain.run({"query": query})
    
    return response.strip()

 ## 5.3. Handle Customer Query

In [ ]:
def handle_customer_query(customer_query):
    """
    Handles the customer query, processes it using the customer service chain,
    and provides the relevant response for the application.
    """
    # Classify if the query is related to an application
    prompt_template = """
    Please classify whether the following query is asking about an application.
    If it is, return 'yes'. If not, return 'no'.
    Customer query: '{customer_query}'
    """
    prompt = PromptTemplate(input_variables=["customer_query"], template=prompt_template)
    application_classification_chain = LLMChain(prompt=prompt, llm=llm)
    is_app_query = application_classification_chain.run({"customer_query": customer_query}).strip().lower()

    # If the query is asking about an application, extract application ID
    if is_app_query == "yes":
        application_id = extract_application_id(customer_query)
        
        if application_id == 'Missing':
            full_response = customer_service_chain.run(customer_query)
        else:
            # Query qa_chain to retrieve the relevant information
            filtered_query = f"Application ID: {application_id}. What is the status of my application and why?"
            retrieved_data = qa_chain.run(filtered_query)
            
            # Combine the initial response with the retrieved data 
            if retrieved_data:
                full_response = f"Thank you for providing your application ID. Here is the specific information for Application ID {application_id}: {retrieved_data}"
            else:
                full_response = customer_service_chain.run(customer_query)
    else:
        # If it doesn't ask about the application
        full_response = customer_service_chain.run(customer_query)
    
    # Update memory 
    memory.save_context({"input": customer_query}, {"output": full_response})

    return full_response

 ## 6. Testing the Chatbot

In [ ]:
# Sample queries
print(handle_customer_query("Why was my application declined?"))
print(handle_customer_query("my application id is 2"))
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
print(handle_customer_query("Hi, my name is andy"))
print(handle_customer_query("I lost my credit card"))
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
print(handle_customer_query("I am not happy"))
print(handle_customer_query("Why my application was declined. My application_id is 3"))
memory.load_memory_variables({})
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
print(handle_customer_query("Hi, my name is andy"))
print(handle_customer_query("What is my name?"))

Hello, thank you for reaching out to us. In order to provide you with more information about why your application was declined, could you please provide me with your application ID? This will help me locate the correct information for you. If you don't have the application ID handy, I recommend contacting our customer support team for further assistance. Thank you for your understanding.


> Entering new RetrievalQA chain...

> Finished chain.
Thank you for providing your application ID. Here is the specific information for Application ID 2: The status of your application with ID 2 is declined. The reason for the decline is "Applicant UC score greater than risk cut-off; Affordability < 0."
Hello Andy, thank you for reaching out to us. How can I assist you today? If you have a specific question or concern regarding your application or account, please provide me with your application ID so that I can locate the necessary information for you.
Thank you for reaching out to us. I'm sorry to